In [2]:
import re
import pandas as pd
from keras.models import load_model
import numpy as np
import pickle
import json

Using TensorFlow backend.


In [87]:
filepath = 'test_data/right.txt'   



In [88]:
clean_data = cleandata(filepath)
lines , data = jaw_clench_lines(clean_data)
final_array = useless_data_removal(data, lines, filepath)

print("Data has been cleaned!")

dfObj = pd.DataFrame(final_array, columns = ['Alpha_1','Alpha_2','Alpha_3','Alpha_4','Beta_1','Beta_2','Beta_3','Beta_4','Delta_1','Delta_2','Delta_3','Delta_4','Theta_1','Theta_2','Theta_3','Theta_4','Gamma_1','Gamma_2','Gamma_3','Gamma_4'])
print('Data has been saved into a dataframe!')
#print(dfObj)

model = load_model('model.h5')

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classes = model.predict_classes(dfObj, batch_size=5)
probability = model.predict(dfObj,batch_size=5)




with open('rf_model.pkl', 'rb') as f:
    rf = pickle.load(f)
    
    rf_preds = rf.predict(dfObj)
    

with open('svm_model.pkl', 'rb') as f:
    svm = pickle.load(f)
    
    svm_preds = svm.predict(dfObj)
    
    
print('predictions from Neural Network: ', classes)
print('predictions from Random Forest: ', rf_preds)
#print('predictions from SVM: ', svm_preds)


results(classes, rf_preds, svm_preds, dfObj)


287
Data has been cleaned!
Data has been saved into a dataframe!


AttributeError: 'list' object has no attribute 'shape'

In [4]:
def cleandata(filepath):
    with open(filepath, 'r') as f2:
                raw_data = f2.read()
                clean_data = raw_data.replace("[8000]", "\n")
                
    return clean_data

In [5]:
def jaw_clench_lines(clean_data):
    
    lines = []
    data = clean_data.split('\n') #split string into a list
    for i,temp in enumerate(data):
        if 'jaw_clench' in temp:
            #print(i)
            lines.append(i)
            
    
    print(len(data))
            
            
    return lines, data

In [6]:
def useless_data_removal(data,lines, filepath):
    
    N = 20
    
    #del data[(lines[2]):]
    del data[:lines[1]]
            
 
            
    gh = "\n".join([str(elem) for elem in data])
    
    data = gh.replace("-∞", "0.00")                 
    data = re.findall("\d+\.\d+", data)
    
   
    subList = [data[n:n+N] for n in range(0, len(data), N)]
    
    return subList

In [66]:
def results(classes, rf_preds , svm_preds, dfObj):
    
    #counts the number of 1s (left predictions)
    left = np.count_nonzero(classes == 1)  + np.count_nonzero(rf_preds == 1) 
    #counts the number of 0s (right predictions)
    right = np.count_nonzero(classes == 0)  + np.count_nonzero(rf_preds == 0)
    length = len(dfObj) #length of the dataset
    accuracy_left = left / (length*2) # average score for left
    accuracy_right = right /(length*2)# average score for right
    
    print("percentage prediction for Left is ", accuracy_left)
    print("percentage prediction for Right is ", accuracy_right)
    
    with open('direction.json' , 'w') as g:
        if accuracy_left > 0.6: #if the accuracy is above 60% then save the direction as L to the json 
            json.dump('L' , g)
            print('The direction predicted is : Left')
        elif accuracy_right > 0.6:#if the accuracy is above 60% then save the direction as R to the json
            json.dump('R' , g)
            print('The direction predicted is : Right')

In [24]:
np.count_nonzero(classes == 0) + np.count_nonzero(rf_preds == 0)

12

In [70]:
len(dfObj)

49

In [77]:
nn_left = np.count_nonzero(classes == 1) #count for predicted left NN
nn_right = np.count_nonzero(classes == 0)#count for predicted right NN

print(nn_left,nn_right)


38 7


In [80]:
rf_left = np.count_nonzero(rf_preds == 1) #count for predicted left rf
rf_right = np.count_nonzero(rf_preds == 0)#count for predicted right rf

print(rf_left,rf_right)

4 41
